In [1]:
!pip install -q anthropic
!pip install -q openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 5.0 MB/s eta 0:00:00


In [2]:
import anthropic
import openai
import json
import pandas as pd
from google.colab import userdata
from google import genai

In [3]:
api_key = userdata.get('ANTHROPIC_API_KEY')
client = anthropic.Anthropic(api_key=api_key)

openai_api_key = userdata.get('OPENAI_API_KEY')
openai_client = openai.OpenAI(api_key = openai_api_key)

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
google_client = genai.Client(api_key=GOOGLE_API_KEY)

In [4]:
def claude_prompt(question, follow_up):
    prompt = f"""You are an expert in natural language processing. Please label the following question-follow-up pair as "1" if the follow-up asks the same or a similar question to the initial question, or if it explicitly or implicitly states that the question was not answered. Otherwise, label "0". It is okay to label 0 if the follow-up is asking for futher clarification or specifics.

Initial question: "{question}"
Follow-up: "{follow_up}"

Provide your label (1 or 0):"""
    return prompt

def claude_label(question, follow_up):
    message = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=150,
        temperature=0,
        system="You analyze question-follow-up pairs and determine if the follow-up is asking the same/similar question or indicating the initial question wasn't answered (label '1'), or if it's a new topic or asking for further details (label '0'). Respond with ONLY the label (1 or 0).",
        messages=[
            {"role": "user", "content": claude_prompt(question, follow_up)}
        ]
    )

    response = message.content[0].text
    return response


def openai_label(question, follow_up):
    prompt = f"Label the following question-follow-up pair as '1' if the follow-up repeats the question or suggests the question was not answered. Otherwise, label '0'.\n\n. It is okay to label 0 if the follow-up is asking for futher clarification or specifics, or if the follow-up is unrelated."
    prompt += f"Question: {question}\nFollow-up: {follow_up}\nLabel:"

    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an NLP model trained to label question-follow-up pairs."},
            {"role": "user", "content": prompt}
        ]
    )

    label = response.choices[0].message.content.strip()
    return label


def google_label(question, follow_up):
    prompt = f"Label the following question-follow-up pair as '1' if the follow-up repeats the question or suggests the question was not answered. Otherwise, label '0'.\n\n. It is okay to label 0 if the follow-up is asking for futher clarification or specifics, or if the follow-up is unrelated."
    prompt += f"Question: {question}\nFollow-up: {follow_up}\nLabel:"

    response = google_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
      )

    return response.text[0]


In [7]:
with open('./drive/MyDrive/w266/project/interview_npr.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [9]:
df["label_claude"] = df.apply(lambda row: claude_label(row["question"], row["follow_up_question"]), axis=1)


In [20]:
df.to_csv('claude_checkpoint')

In [29]:
df["label_openai"] = df.apply(lambda row: openai_label(row["question"], row["follow_up_question"]), axis=1)


In [30]:
df.to_csv('openai_checkpoint')

In [33]:
df["label_google"] = df.apply(lambda row: google_label(row["question"], row["follow_up_question"]), axis=1)


In [34]:
df.to_csv('google_checkpoint')

In [35]:
df

,interviewer,interviewee,question,response,follow_up_question,label_claude,label_openai,label_google
0,"LULU GARCIA-NAVARRO, HOST","SUSAN DAVIS, BYLINE",All right. Is this past week a taste of what's...,"You know, it's certainly part of President Tru...","You say that this is Trump's only path, sort o...",0,0,0
1,"NEAL CONAN, HOST",TODD HEISLER,"Todd Heisler, you were the person who was ther...",Yeah. That was the most important part for me ...,We have a link to the Final Salute series at o...,0,0,0
2,"DAVID GREENE, HOST","SCOTT DETROW, BYLINE",OK. So the answer is yes.,Yes (laughter).,You did get a little time to enjoy the fair. N...,0,0,0
3,"LULU GARCIA-NAVARRO, HOST",DAN ANGOTTI,Oh really? It's called Freedom? That's great.,Yup.,How did it get that name?,0,0,0
4,"NEAL CONAN, HOST",SHARON,"Well, let's see if we can get some callers in ...","Yes, hi.",Hi.,0,1,0
...,...,...,...,...,...,...,...,...
4995,"MADELEINE BRAND, host",_NO_SPEAKER,This is Day to Day from NPR News. I'm Madelein...,(Soundbite of hammering),"Madame Speaker, let me present the want-to-be ...",0,0,0
4996,"IRA FLATOW, HOST",CATHERINE LA FARGE,"Yeah, about the mosses that you found. Are all...",The mosses that we've been able to rejuvenate ...,Is it possible to go out in your backyard and ...,0,0,0
4997,"LYNN NEARY, HOST",AMY BOESKY,"So did you get to come up with the plot, the d...",We had. We - so ghostwriters - and I was - for...,"So, like, can you give an example of what this...",0,0,0
4998,"JENNIFER LUDDEN, HOST",ALICIA PARTNOY,But he didn't go through nearly what you did. ...,"It's been my struggle all these years, first, ...","And we should remind the listeners, there were...",0,0,0


In [38]:
df[df['label_claude']==df["label_openai"]]

,interviewer,interviewee,question,response,follow_up_question,label_claude,label_openai,label_google
0,"LULU GARCIA-NAVARRO, HOST","SUSAN DAVIS, BYLINE",All right. Is this past week a taste of what's...,"You know, it's certainly part of President Tru...","You say that this is Trump's only path, sort o...",0,0,0
1,"NEAL CONAN, HOST",TODD HEISLER,"Todd Heisler, you were the person who was ther...",Yeah. That was the most important part for me ...,We have a link to the Final Salute series at o...,0,0,0
2,"DAVID GREENE, HOST","SCOTT DETROW, BYLINE",OK. So the answer is yes.,Yes (laughter).,You did get a little time to enjoy the fair. N...,0,0,0
3,"LULU GARCIA-NAVARRO, HOST",DAN ANGOTTI,Oh really? It's called Freedom? That's great.,Yup.,How did it get that name?,0,0,0
5,"ARI SHAPIRO, HOST",ANGELA,Sure.,"But the director, Dr. Molina(ph), told us from...","And did that work, that he was able to conduct...",0,0,1
...,...,...,...,...,...,...,...,...
4995,"MADELEINE BRAND, host",_NO_SPEAKER,This is Day to Day from NPR News. I'm Madelein...,(Soundbite of hammering),"Madame Speaker, let me present the want-to-be ...",0,0,0
4996,"IRA FLATOW, HOST",CATHERINE LA FARGE,"Yeah, about the mosses that you found. Are all...",The mosses that we've been able to rejuvenate ...,Is it possible to go out in your backyard and ...,0,0,0
4997,"LYNN NEARY, HOST",AMY BOESKY,"So did you get to come up with the plot, the d...",We had. We - so ghostwriters - and I was - for...,"So, like, can you give an example of what this...",0,0,0
4998,"JENNIFER LUDDEN, HOST",ALICIA PARTNOY,But he didn't go through nearly what you did. ...,"It's been my struggle all these years, first, ...","And we should remind the listeners, there were...",0,0,0


In [39]:
df[df['label_claude']==df["label_google"]]

,interviewer,interviewee,question,response,follow_up_question,label_claude,label_openai,label_google
0,"LULU GARCIA-NAVARRO, HOST","SUSAN DAVIS, BYLINE",All right. Is this past week a taste of what's...,"You know, it's certainly part of President Tru...","You say that this is Trump's only path, sort o...",0,0,0
1,"NEAL CONAN, HOST",TODD HEISLER,"Todd Heisler, you were the person who was ther...",Yeah. That was the most important part for me ...,We have a link to the Final Salute series at o...,0,0,0
2,"DAVID GREENE, HOST","SCOTT DETROW, BYLINE",OK. So the answer is yes.,Yes (laughter).,You did get a little time to enjoy the fair. N...,0,0,0
3,"LULU GARCIA-NAVARRO, HOST",DAN ANGOTTI,Oh really? It's called Freedom? That's great.,Yup.,How did it get that name?,0,0,0
4,"NEAL CONAN, HOST",SHARON,"Well, let's see if we can get some callers in ...","Yes, hi.",Hi.,0,1,0
...,...,...,...,...,...,...,...,...
4995,"MADELEINE BRAND, host",_NO_SPEAKER,This is Day to Day from NPR News. I'm Madelein...,(Soundbite of hammering),"Madame Speaker, let me present the want-to-be ...",0,0,0
4996,"IRA FLATOW, HOST",CATHERINE LA FARGE,"Yeah, about the mosses that you found. Are all...",The mosses that we've been able to rejuvenate ...,Is it possible to go out in your backyard and ...,0,0,0
4997,"LYNN NEARY, HOST",AMY BOESKY,"So did you get to come up with the plot, the d...",We had. We - so ghostwriters - and I was - for...,"So, like, can you give an example of what this...",0,0,0
4998,"JENNIFER LUDDEN, HOST",ALICIA PARTNOY,But he didn't go through nearly what you did. ...,"It's been my struggle all these years, first, ...","And we should remind the listeners, there were...",0,0,0


In [40]:
df[df['label_openai']==df["label_google"]]

,interviewer,interviewee,question,response,follow_up_question,label_claude,label_openai,label_google
0,"LULU GARCIA-NAVARRO, HOST","SUSAN DAVIS, BYLINE",All right. Is this past week a taste of what's...,"You know, it's certainly part of President Tru...","You say that this is Trump's only path, sort o...",0,0,0
1,"NEAL CONAN, HOST",TODD HEISLER,"Todd Heisler, you were the person who was ther...",Yeah. That was the most important part for me ...,We have a link to the Final Salute series at o...,0,0,0
2,"DAVID GREENE, HOST","SCOTT DETROW, BYLINE",OK. So the answer is yes.,Yes (laughter).,You did get a little time to enjoy the fair. N...,0,0,0
3,"LULU GARCIA-NAVARRO, HOST",DAN ANGOTTI,Oh really? It's called Freedom? That's great.,Yup.,How did it get that name?,0,0,0
6,"IRA FLATOW, HOST","FLORA LICHTMAN, BYLINE",Right. Right.,"And so he's - you know, we actually talked to ...",Wow. Wow.,0,0,0
...,...,...,...,...,...,...,...,...
4995,"MADELEINE BRAND, host",_NO_SPEAKER,This is Day to Day from NPR News. I'm Madelein...,(Soundbite of hammering),"Madame Speaker, let me present the want-to-be ...",0,0,0
4996,"IRA FLATOW, HOST",CATHERINE LA FARGE,"Yeah, about the mosses that you found. Are all...",The mosses that we've been able to rejuvenate ...,Is it possible to go out in your backyard and ...,0,0,0
4997,"LYNN NEARY, HOST",AMY BOESKY,"So did you get to come up with the plot, the d...",We had. We - so ghostwriters - and I was - for...,"So, like, can you give an example of what this...",0,0,0
4998,"JENNIFER LUDDEN, HOST",ALICIA PARTNOY,But he didn't go through nearly what you did. ...,"It's been my struggle all these years, first, ...","And we should remind the listeners, there were...",0,0,0


In [41]:
def choose_label(row):
    if row["label_claude"] == row["label_google"]:
        return row["label_claude"]
    else:
        return row["label_openai"]

In [42]:
df["label"] = df.apply(lambda row: choose_label(row), axis=1)

In [43]:
df[df['label']=='1']

,interviewer,interviewee,question,response,follow_up_question,label_claude,label_openai,label_google,label
26,"AILSA CHANG, HOST",BLACK BELT EAGLE SCOUT,"So if you had it your way, would you not want ...",No. I think that I would want them to be able ...,"So again, like, if you were to make up the rul...",1,1,1,1
119,"JOHN DONVAN, HOST",ANITA BRUZZESE,Let's take just a minute to define our terms a...,"Well, I think one of the things we do need to ...",And an extrovert is what?,0,1,1,1
323,"NEAL CONAN, HOST",BENJAMIN,"Well, maybe you're making different choices wh...","At times, I suppose. But then I still usually ...","OK, so making one of those non-statements.",1,1,0,1
334,"NEAL CONAN, HOST",_NO_SPEAKER,We're talking with doctors about how they make...,(SOUNDBITE OF MUSIC),This is TALK OF THE NATION from NPR News; I'm ...,1,1,1,1
362,"LULU GARCIA-NAVARRO, HOST",IVAN MERKENICH,Sretno.,Sretno.,Sretno. OK. Sretno. Thank you so much.,0,1,1,1
...,...,...,...,...,...,...,...,...,...
4809,"IRA FLATOW, HOST",IAN MCEWAN,"Yeah, you can understand what goes on?","I can read the abstracts, yeah.",Yeah?,0,1,1,1
4829,"NEAL CONAN, HOST",CARRIE,We want to get callers in on the conversation....,Hi there.,"Hi, Carrie. How did you find out about the job...",1,1,1,1
4844,"CELESTE HEADLEE, HOST",TANISHIA WILLIAMS-MINOR,To what end?,..that's not typical. To what end...,To what end? What's the benefit of them seeing...,1,1,1,1
4904,"ARI SHAPIRO, HOST",ADAM SCHIFF,What about the Ukrainian president's statement...,"Well, first of all, he clearly did feel pressu...",Even if he says he didn't.,1,1,1,1


In [44]:
df.to_csv('labeled_data_npr.csv')